
# Image Processing SS 18 - Assignment - 10

### Deadline is 27.06.2016 at 8:00 a.m. 

Please solve the assignments together with a partner.
I will run every notebook. Make sure the code runs through. Select `Kernel` -> `Restart & Run All` to test it.
Please strip the output from the cells, either select `Cell` -> `All Output` -> `Clear` or use the `nb_strip_output.py` script / git hook.

In [ ]:
# display the plots inside the notebook
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pylab
import scipy.io.wavfile
from skimage.data import astronaut
from skimage.color import rgb2gray
from skimage.transform import rescale, resize, downscale_local_mean

from __future__ import division
import random
try:
    from StringIO import StringIO as BytesIO
except ImportError:
    from io import BytesIO
    
try:
    import urllib.request as urllib2
except ImportError:
    import urllib2
    
    
from numpy.fft import fft2 as numpy_fft2, ifft2 as numpy_ifft2
from scipy.fftpack import dctn, idctn
from PIL import Image
import itertools
import IPython
import zipfile
pylab.rcParams['figure.figsize'] = (12, 12)   # This makes the plot bigger

# Exercise 1 - Hadamard Matrix - 5 Points



In [ ]:
def hadamard_matrix(n):
    """Returns the Hadamard matrix. N is a power of two."""
    n = int(n)
    assert (n > 0) and ((n & (n - 1)) == 0), "argument has to be a power of two!"
    H = np.zeros((n,n))
    H[0,0] = 1
    for i in range(int(np.log2(n))):
        rec = H[0:2**i,0:2**i]
        H[2**i:2**(i+1), 0:2**i] = rec
        H[0:2**i, 2**i:2**(i+1)] = rec
        H[2**i:2**(i+1), 2**i:2**(i+1)] = -1*rec
    return H / np.sqrt(n)

plt.subplot(121)
plt.imshow(hadamard_matrix(8), cmap='gray')
plt.subplot(122)
plt.imshow(hadamard_matrix(32), cmap='gray')
plt.show()

In [ ]:
def get_chess_board(n=8, field_size=8):
    board = np.zeros((n*field_size, n*field_size))
    s = field_size
    for i in range(n):
        for j in range(n):
            if (i + j) % 2 == 0:
                board[i*s:(i+1)*s, j*s:(j+1)*s] = 1
    return board

def get_sinus_board(n=8, field_size=8, shift=0):
    img_size = n*field_size
    linsp = np.linspace(-shift, np.pi*n - shift, img_size).reshape((img_size, 1))
    return np.dot(np.sin(linsp), np.sin(linsp.T))


In [ ]:
chess_board = get_chess_board()
chess_board_roll = np.roll(np.roll(chess_board, 4, axis=0), 4, axis=1)
sinus_board = get_sinus_board()
sinus_board_roll = get_sinus_board(shift=4)
plt.subplot(221)
plt.imshow(chess_board, cmap='gray')
plt.subplot(222)
plt.imshow(sinus_board, cmap='gray')
plt.subplot(223)
plt.imshow(chess_board_roll, cmap='gray')
plt.subplot(224)
plt.imshow(sinus_board_roll, cmap='gray')
plt.show()

In [ ]:
# Plot the 2d hadamard transformation of the chess boards and sinus boards.
# The 2D Hadamard Transformation can be computed similiar to the 2D DFT:
# H * I * H, where I is the image, H is the hadamard-matrix and * is the matrix multiplication.

H = hadamard_matrix(64)

chess_board_H = (H @ chess_board) @ H
chess_board_roll_H = (H @ chess_board_roll) @ H
sinus_board_H = (H @ sinus_board) @ H
sinus_board_roll_H = (H @ sinus_board_roll) @ H

plt.subplot(221)
plt.imshow(chess_board_H, cmap='gray')
plt.subplot(222)
plt.imshow(sinus_board_H, cmap='gray')
plt.subplot(223)
plt.imshow(chess_board_roll_H, cmap='gray')
plt.subplot(224)
plt.imshow(sinus_board_roll_H, cmap='gray')
plt.show()

In [ ]:
plt.imshow(np.dot(np.dot(H, chess_board_roll), H))

# Plot the fourier transformation of chess boards and sinus boards. 
# You can use some functions from np.ftt
chess_board_ft = np.real(numpy_fft2(chess_board))
chess_board_roll_ft = np.real(numpy_fft2(chess_board_roll) )
sinus_board_ft = np.real(numpy_fft2(sinus_board))
sinus_board_roll_ft = np.real(numpy_fft2(sinus_board_roll))

plt.subplot(221)
plt.imshow(chess_board_ft, cmap='gray')
plt.subplot(222)
plt.imshow(sinus_board_ft, cmap='gray')
plt.subplot(223)
plt.imshow(chess_board_roll_ft, cmap='gray')
plt.subplot(224)
plt.imshow(sinus_board_roll_ft, cmap='gray')
plt.show()

# Exercise 2 - Compare DCT with Discrete Hadamard Transformation - 5 Points

Load the astronaut image, make sure it has a resolution of 256*256 pixels. 

Transform with :

a) DCT

b) DiskreteWalshHadamardTransf

Use different window sizes: 8x8, 32x32, 256x256. 

Erase **75 percent** of coefficents. 

Transform back into image space and calculate the average quadratic pixel error w.r.t. the original image. 

Make a table, which contains on top: DCT / DWalHadT and shows the error for WindowSize 8x8, 32x32, 256x256 
for DCT and DWalHadT respectively.

In [ ]:
def dwht2(img):
    assert img.shape[0] == img.shape[1]
    transformed = np.copy(img)
    M = hadamard_matrix(img.shape[0])
    return (M @ transformed) @ M

In [ ]:
def idwht2(img):
    assert img.shape[0] == img.shape[1]
    transformed = np.copy(img)
    M = np.linalg.inv(hadamard_matrix(img.shape[0]))
    return (M @ transformed) @ M

In [ ]:
def dct2(img):
    return dctn(img, norm='ortho')

In [ ]:
def idct2(img):
    return idctn(img, norm='ortho')

In [ ]:
def eraseCoeff(transformation, amount=0.25):
    '''erases (approx.) amount*100% of coefficents of transformation (sets them 0)'''
    assert 1 >= amount and amount >= 0, "amount has to be a fraction (float, element of [0,1])"
    total = np.prod(transformation.shape[0:2])
    side = int(round(np.sqrt(total*(1-amount))))
    mask = np.zeros_like(transformation)
    mask[:side, :side] = 1
    return transformation * mask  

In [ ]:
def apply_transform(data, window_size, func, erase=False):
    assert (data.shape[0] % window_size) == 0 and (data.shape[1] % window_size) == 0
    transformed = np.zeros_like(data)
    for i in range(0, data.shape[0], window_size):
        for j in range(0, data.shape[1], window_size):
            trans = func(data[i:i+window_size, j:j+window_size])
            if erase:
                trans = eraseCoeff(trans, 0.75)
            transformed[i:i+window_size, j:j+window_size] = trans
    return transformed 

In [ ]:
astro = rgb2gray(astronaut())

In [ ]:
print(np.min(astro), np.max(astro))

In [ ]:
# No downscaling, 'cause Prof. said so

In [ ]:
#astro = downscale_local_mean(astro, (2, 2))

In [ ]:
astro.shape

In [ ]:
plt.imshow(astro, cmap='gray')
plt.show()

In [ ]:
window_sizes = [2**i for i in range(1, int(np.log2(astro.shape[0]))+1)]
window_sizes

In [ ]:
DCT = [apply_transform(astro, win_size, dct2, erase=True) for win_size in window_sizes]
for dct_transform in DCT:
    plt.imshow(dct_transform, cmap='gray')
    plt.show()

In [ ]:
DWHT = [apply_transform(astro, win_size, dwht2, erase=True) for win_size in window_sizes]
for dwht_transform in DWHT:
    plt.imshow(dwht_transform, cmap='gray')
    plt.show()

In [ ]:
DCT_recon = [apply_transform(dct, win_size, idct2) for (dct, win_size) in zip(DCT, window_sizes)]
for dct_rec in DCT_recon:
    plt.imshow(dct_rec, cmap='gray')
    plt.show()

In [ ]:
DWHT_recon = [apply_transform(dwht, win_size, idwht2) for (dwht, win_size) in zip(DWHT, window_sizes)]
for dwht_rec in DWHT_recon:
    plt.imshow(dwht_rec, cmap='gray')
    plt.show()

In [ ]:
np.min(DWHT_recon), np.max(DWHT_recon)

In [ ]:
np.min(DCT_recon), np.max(DCT_recon)

In [ ]:
def norm_img(img):
    if np.min(img) < 0:
        img += np.abs(np.min(img))
    img /= np.max(img)
    return img

In [ ]:
for dct_rec in DCT_recon:
    norm_img(dct_rec)

In [ ]:
for dwht_rec in DWHT_recon:
    norm_img(dwht_rec)

In [ ]:
def diff(X, Y):
    assert X.shape == Y.shape
    return np.sum(np.power((X-Y), 2)) / np.prod(X.shape[:2])

In [ ]:
DCT_diffs = [diff(astro, dct_rec) for dct_rec in DCT_recon]
DWHT_diffs = [diff(astro, dwht_rec) for dwht_rec in DWHT_recon]

In [ ]:
data = np.array([DCT_diffs, DWHT_diffs]).T
data = np.round(data, 5)
root_data = np.sqrt(np.copy(data))
title = "Reconstruction error after 75 percent of coefficients erased"

In [ ]:
columns = ('DCT', 'DWalHadT')
rows = ['%dx%d' % (w_size, w_size) for w_size in window_sizes]

In [ ]:
plot_DCT = DCT_diffs
plot_DWHT = DWHT_diffs
ceil = 0.1

In [ ]:
plot_DCT = np.sqrt(np.copy(DCT_diffs))
plot_DWHT = np.sqrt(np.copy(DWHT_diffs))
ceil = 0.3

In [ ]:
pylab.rcParams['figure.figsize'] = (14, 8)
plt.style.use('ggplot')

In [ ]:
plt.plot(plot_DCT, 'r')
plt.plot(plot_DCT, 'ro')
plt.plot(plot_DWHT, 'b')
plt.plot(plot_DWHT, 'bo')
plt.axis([0, len(rows), 0, ceil])
locs, labels = plt.xticks()
plt.xticks(locs, rows)
plt.title(title)
plt.xlabel('Subimage size', fontsize=14)
plt.ylabel('Root-mean-square error', fontsize=14)
plt.text(len(rows)-0.9, plot_DCT[-1], columns[0], fontsize=14)
plt.text(len(rows)-0.9, plot_DWHT[-1], columns[1], fontsize=14)
plt.show()

In [ ]:
import pandas as pd
df = pd.DataFrame(data, columns=columns, index=rows)

In [ ]:
print(title,"\n-> Mean-square error depending on subimage size in pixels")
df